<a href="https://colab.research.google.com/github/minyansh7/Terramare-AudienceResearch/blob/main/Insights_listening_Reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded = files.upload()


Saving Insights_listening_Reddit.ipynb to Insights_listening_Reddit.ipynb


**Section 1. Reddit Data Scrape**



In [ ]:
# 🛠️ Install dependencies
!pip install praw pandas

import praw
import re
import pandas as pd
from datetime import datetime, timedelta
import logging

In [ ]:
# prompt: set Reddit_Client_ID
# Reddit_Secret in secrets, how to read it in colab

from google.colab import userdata
reddit_client_id = userdata.get('Reddit_Client_ID')
reddit_secret = userdata.get('Reddit_Secret')
user_agent = 'd-listening/0.1'

# Suppress PRAW's async environment warning
logging.getLogger("praw").setLevel(logging.ERROR)


In [ ]:
import praw
import re
import logging
import pandas as pd
from datetime import datetime, timedelta

# === ✅ Setup Reddit connection ===
reddit = praw.Reddit(
    client_id=reddit_client_id,
    client_secret=reddit_secret,
    user_agent=user_agent
)

# === 🎯 Subreddits to scan ===
aussie_subs = [
    "meditation", "streamentry", "Buddhism", "Zen",
    "biohackers", "flowarts", "selfimprovement"
]

# === 🇦🇺 Aussie context keywords (normalized) ===
aussie_terms = [t.lower() for t in [
    "australia", "aussie", "nsw", "vic", "centrelink", "medicare", "mygov",
    "woolies", "coles", "uni", "headspace", "black dog", "beyond blue",
    "tafe", "smiling mind", "r u ok", "NDIS", "QLD", "ACT", "myki", "VIC Health"
]]

# === 🧘 Meditation term pattern (fuzzy) ===
meditation_pattern = re.compile(r"\bmeditat(?:ion|ing|e)?\b", re.IGNORECASE)

# === 🔎 Scraping setup ===
max_items = 5000
scraped_count = 0
results = []

# === 🕒 Time filter for last 2 years ===
two_years_ago = datetime.utcnow() - timedelta(days=730)
two_years_ago_timestamp = int(two_years_ago.timestamp())

# === 🔁 Phase 1: Scrape Posts ===
logging.info("Phase 1: Scraping Posts...")
for sub in aussie_subs:
    if scraped_count >= max_items:
        break
    logging.info(f"🔍 Scanning r/{sub}...")

    try:
        for post in reddit.subreddit(sub).new(limit=5000):
            if scraped_count >= max_items:
                break
            if post.created_utc < two_years_ago_timestamp or post.score < 2:
                continue

            content = f"{post.title} {post.selftext}".lower()
            flair = (post.link_flair_text or "").lower()
            user_flair = (post.author_flair_text or "").lower()
            has_meditation_term = meditation_pattern.search(content)
            matched_aussie_terms = [
                term for term in aussie_terms
                if term in content or term in flair or term in user_flair
            ]
            if has_meditation_term and matched_aussie_terms:
                results.append({
                    "type": "post",
                    "subreddit": post.subreddit.display_name,
                    "author": str(post.author),
                    "score": post.score,
                    "title": post.title,
                    "full_text": post.selftext[:4000],
                    "matched_terms": matched_aussie_terms,
                    "url": f"https://reddit.com{post.permalink}",
                    "created_utc": datetime.utcfromtimestamp(post.created_utc).isoformat(),
                    "link_flair_text": post.link_flair_text,
                    "num_comments": post.num_comments,
                    "id": post.id
                })
                scraped_count += 1

    except Exception as e:
        logging.error(f"Error scraping r/{sub} posts: {e}")

# === 💬 Phase 2: Scrape Comments from Found Posts ===
logging.info("\nPhase 2: Scraping Comments from Found Posts...")
post_ids_to_check_comments = [r['id'] for r in results if r['type'] == 'post']

for post_id in post_ids_to_check_comments:
    if scraped_count >= max_items:
        break

    try:
        submission = reddit.submission(id=post_id)
        submission.comments.replace_more(limit=0)
        all_comments = submission.comments.list()
        logging.info(f"  Checking {len(all_comments)} comments in: {submission.title[:50]}...")

        for comment in all_comments:
            if scraped_count >= max_items:
                break
            if comment.created_utc < two_years_ago_timestamp or comment.score < 2:
                continue

            body = comment.body.lower()
            has_meditation_term = meditation_pattern.search(body)
            matched_aussie_terms = [term for term in aussie_terms if term in body]

            if has_meditation_term and matched_aussie_terms:
                results.append({
                    "type": "comment",
                    "subreddit": comment.subreddit.display_name,
                    "author": str(comment.author),
                    "score": comment.score,
                    "title": None,
                    "full_text": comment.body[:4000],
                    "matched_terms": matched_aussie_terms,
                    "url": f"https://reddit.com{comment.permalink}",
                    "created_utc": datetime.utcfromtimestamp(comment.created_utc).isoformat(),
                    "link_flair_text": None,
                    "comment_depth": getattr(comment, "depth", None)
                })
                scraped_count += 1

    except Exception as e:
        logging.error(f"Error scraping comments for post {post_id}: {e}")

# === 📊 Output Results ===
df = pd.DataFrame(results).head(max_items)
pd.set_option('display.max_colwidth', None)

print(f"\n✅ Scraped {len(df)} AU-relevant items mentioning meditation (target: {max_items}):\n")
print(df.head(10))

# Stats
print(f"\nTotal items scraped: {len(df)}")
print(f"Number of posts scraped: {len(df[df['type'] == 'post'])}")
print(f"Number of comments scraped: {len(df[df['type'] == 'comment'])}")
print(f"Median Score: {df['score'].median()}")
print(f"Average Score: {df['score'].mean():.2f}")
print(f"Items by Subreddit:\n{df['subreddit'].value_counts()}")



✅ Scraped 2977 AU-relevant items mentioning meditation (target: 5000):

   type   subreddit                author  score  \
0  post  Meditation          Anima_Monday      2   
1  post  Meditation           FunnyOWL007      7   
2  post  Meditation           kikemeister      2   
3  post  Meditation              TinaTeng      9   
4  post  Meditation                sidgat      8   
5  post  Meditation  Intrepid_Strike_2454      2   
6  post  Meditation    ExperienceMany4417     35   
7  post  Meditation         EthanGisclair      6   
8  post  Meditation        AnnualPath9528      5   
9  post  Meditation         Teastainedeye      8   

                                                                                                       title  \
0                                                 Equanimity and Non-Attachment Meditation, A Novel Approach   
1                                       I didn’t think meditation could help the planet… until I tried this.   
2                 

In [ ]:
# ✅ Setup Reddit connection
######
######
######
######
#original script#replaced by above optimised one
reddit = praw.Reddit(
    client_id=reddit_client_id,
    client_secret=reddit_secret,
    user_agent=user_agent
)
# 🎯 Subreddits to scan
aussie_subs = [
    "meditation",       # Core subreddit
    "streamentry",      # Hardcore Buddhist meditation community
    "Buddhism",         # Vipassana and insight posts
    "Zen",               # R/zen — Koans, concentration, awareness
    "biohackers",       # NSDR, nootropics, breathwork
    "flowarts",         # Movement meditation, flow
    "selfimprovement"   # Habit-building overlap
]


# === 🇦🇺 Aussie context keywords (normalized) ===
aussie_terms = [t.lower() for t in [
    "australia", "aussie", "nsw", "vic", "centrelink", "medicare", "mygov",
    "woolies", "coles", "uni", "headspace", "black dog", "beyond blue",
    "tafe", "smiling mind", "r u ok", "NDIS", "QLD", "ACT", "myki", "VIC Health"
]]

# === 🧘 Meditation term pattern (fuzzy) ===
meditation_pattern = re.compile(r"\bmeditat(?:ion|ing|e)?\b", re.IGNORECASE)

# === 🔎 Scraping setup ===
max_items = 2
scraped_count = 0
results = []
# We will handle the date filtering differently depending on the method used

# Set a target timestamp for the last 2 years, but note that methods like .new()
# don't use 'after' with a timestamp like search does. We'll filter later.
# Time threshold
two_years_ago = datetime.utcnow() - timedelta(days=730)
two_years_ago_timestamp = int(two_years_ago.timestamp())

# === 🔁 Scrape Posts First (using .new() with time_filter or .top()) ===
# Trying .new() with 'year' time_filter first
logging.info("Phase 1: Scraping Posts...")
for sub in aussie_subs:
    if scraped_count >= max_items:
        break
    logging.info(f"🔍 Scanning recent posts in r/{sub}...")

    try:
        # Use .new() with time_filter 'year' to get recent posts within the last year
        # Note: This is approximate and doesn't guarantee exactly 2 years. We filter later.
        for post in reddit.subreddit(sub).new(limit=3000): # Increased limit per sub to find more candidates
            if scraped_count >= max_items:
                break

            # Apply time filter (explicitly check timestamp)
            if post.created_utc < two_years_ago_timestamp:
                continue # Skip if older than 2 years


            # Filter criteria: meditation keyword, Aussie context, score >= 2
            content = f"{post.title} {post.selftext}".lower()
            flair = (post.link_flair_text or "").lower()
            user_flair = (post.author_flair_text or "").lower()
            matched_aussie_terms = [term for term in aussie_terms if term in content or term in flair or term in user_flair]
            has_meditation_term = meditation_pattern.search(content) is not None
            meets_score = post.score >= 2

            if has_meditation_term and matched_aussie_terms and meets_score:
                results.append({
                    "type": "post",
                    "subreddit": post.subreddit.display_name,
                    "author": str(post.author),
                    "score": post.score,
                    "title": post.title,
                    "full_text": post.selftext[:4000],
                    "matched_terms": matched_aussie_terms,
                    "url": f"https://reddit.com{post.permalink}",
                    "created_utc": datetime.utcfromtimestamp(post.created_utc).isoformat(),
                    "link_flair_text": post.link_flair_text,
                    "num_comments": post.num_comments,
                    "id": post.id # Store ID to fetch comments later
                })
                scraped_count += 1
                # logging.info(f"  Found post: {post.title} (Score: {post.score}, Scraped: {scraped_count})")

    except Exception as e:
        logging.error(f"Error scraping r/{sub} posts: {e}")


# === 💬 Scrape Comments from the Scraped Posts ===
# This is much more targeted and efficient than scraping all comments in a subreddit.
logging.info("\nPhase 2: Scraping Comments from Found Posts...")
post_ids_to_check_comments = [r['id'] for r in results if r['type'] == 'post'] # Get IDs of found posts

for post_id in post_ids_to_check_comments:
    if scraped_count >= max_items:
        break

    try:
        submission = reddit.submission(id=post_id)
        # Fetch all comments from the submission (Reddit may return a forest, not just top level)
        # Use replace_more(limit=0) to avoid fetching potentially huge amounts of 'MoreComments' objects
        submission.comments.replace_more(limit=0)
        all_comments = submission.comments.list()

        logging.info(f"  Checking {len(all_comments)} comments for post: {submission.title[:50]}...")

        for comment in all_comments:
            if scraped_count >= max_items:
                break

            # Skip if older than 2 years (check comment creation time)
            if comment.created_utc < two_years_ago_timestamp:
                continue # Skip if older than 2 years

            body = comment.body.lower()

            # Filter criteria: meditation keyword, Aussie context, score >= 2
            has_meditation_term = meditation_pattern.search(body) is not None
            matched_aussie_terms = [term for term in aussie_terms if term in body]
            meets_score = comment.score >= 2

            if has_meditation_term and matched_aussie_terms and meets_score:
                results.append({
                    "type": "comment",
                    "subreddit": comment.subreddit.display_name,
                    "author": str(comment.author),
                    "score": comment.score,
                    "title": None, # Comments don't have titles
                    "full_text": comment.body[:4000],
                    "matched_terms": matched_aussie_terms,
                    "url": f"https://reddit.com{comment.permalink}",
                    "created_utc": datetime.utcfromtimestamp(comment.created_utc).isoformat(),
                    "link_flair_text": None, # Comments don't have link flair
                    "comment_depth": getattr(comment, "depth", None)
                })
                scraped_count += 1
                # logging.info(f"    Found comment (Score: {comment.score}, Scraped: {scraped_count})")

    except Exception as e:
        logging.error(f"Error scraping comments for post {post_id}: {e}")

# === 📊 Output results ===
df = pd.DataFrame(results)
# Ensure we only keep the required number of items if we exceeded max_items during the process
df = df.head(max_items)

# Sort by score or another relevant metric to potentially show higher quality results first
#df = df.sort_values(by='score', ascending=False).reset_index(drop=True)


pd.set_option('display.max_colwidth', None)
print(f"\n✅ Scraped {len(df)} AU-relevant items mentioning meditation (target: {max_items}):\n")
print(df.head(10))

# Display some stats
print(f"\nTotal items scraped: {len(df)}")
print(f"Number of posts scraped: {len(df[df['type'] == 'post'])}")
print(f"Number of comments scraped: {len(df[df['type'] == 'comment'])}")
print(f"Median Score: {df['score'].median()}")
print(f"Average Score: {df['score'].mean():.2f}")
print(f"Items by Subreddit:\n{df['subreddit'].value_counts()}")



In [ ]:
# To optimise and hit the 1000 target:
# - Focus scraping on more targeted subreddits first.
# - Use the `subreddit.new()` or `subreddit.controversial()` or `subreddit.top()` methods with a time filter (`time_filter='year'` or `time_filter='all'`) instead of `search` for posts, as these can be more effective for recent activity within a time window.
# - For comments, instead of iterating *all* comments, iterate through the comments of the *scraped posts* that met the initial criteria. This keeps the comment search relevant to the found posts.
# - Increase the limits for initial post scraping to allow more potential candidates before filtering.
# - Implement a more dynamic approach that potentially increases the scrape depth or expands search criteria if the initial pass doesn't yield enough results.
# - Prioritize scoring potentially higher based on keywords or subreddits if needed.

In [ ]:
import duckdb
from datetime import datetime
import pandas as pd

# Generate timestamped filename
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
db_file = f"reddit_{timestamp}.duckdb"
table_name = "reddit_meditation_au"

# Save table
con = duckdb.connect(db_file)
con.execute(f"DROP TABLE IF EXISTS {table_name}")
con.execute(f"CREATE TABLE {table_name} AS SELECT * FROM df")
con.close()

print(f"✅ Saved table '{table_name}' to {db_file}")


CatalogException: Catalog Error: Table with name df does not exist!
Did you mean "pg_depend"?

LINE 1: CREATE TABLE reddit_meditation_au AS SELECT * FROM df
                                                           ^

In [ ]:
con = duckdb.connect(db_file)
tables = con.execute("SHOW TABLES").fetchall()
print("📋 Tables before copy:", tables)
con.close()


📋 Tables before copy: [('reddit_meditation_au',)]


In [5]:
from google.colab import drive
import os
import shutil

# Mount Drive
drive.mount('/content/drive')

# Define paths
drive_path = '/content/drive/MyDrive/reddit_data'
os.makedirs(drive_path, exist_ok=True)

# Copy using same name
dest_file = os.path.join(drive_path, db_file)
shutil.copy(db_file, dest_file)
print(f"✅ Copied '{db_file}' to Drive → {dest_file}")

Mounted at /content/drive
✅ Copied 'reddit_20250606_190637.duckdb' to Drive → /content/drive/MyDrive/reddit_data/reddit_20250606_190637.duckdb


**Section 2. Reload data from Google Drive**

In [2]:
# ✅ Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# ✅ Step 2: Set full path to your DuckDB file
import duckdb

dest_file = "/content/drive/MyDrive/reddit_data/reddit_20250605_083205.duckdb"

# Connect to the file
con = duckdb.connect(dest_file)
# ✅ Step 3: Check available tables
tables = con.execute("SHOW TABLES").fetchall()
print("📋 Tables in Drive copy:", tables)


Mounted at /content/drive
📋 Tables in Drive copy: [('reddit_meditation_au',)]


In [3]:
# Try loading
df_loaded = con.execute("SELECT * FROM reddit_meditation_au").df()
print(f"✅ Loaded {len(df_loaded)} rows from Drive copy.")
con.close()


✅ Loaded 2977 rows from Drive copy.


In [4]:
import pandas as pd
# === 📊 STEP 4: Preview & Summary ===
if not df_loaded.empty:
    pd.set_option('display.max_colwidth', None)
    print(df_loaded.head(10))

    # Summary stats
    print("\n📊 Summary:")
    print(f"- Total items: {len(df_loaded)}")
    print(f"- Posts: {len(df_loaded[df_loaded['type'] == 'post'])}")
    print(f"- Comments: {len(df_loaded[df_loaded['type'] == 'comment'])}")
    print(f"- Median Score: {df_loaded['score'].median()}")
    print(f"- Average Score: {df_loaded['score'].mean():.2f}")
    print("\n🔎 Items by Subreddit:")
    print(df_loaded['subreddit'].value_counts())
else:
    print("No data loaded.")


   type   subreddit                author  score  \
0  post  Meditation          Anima_Monday      2   
1  post  Meditation           FunnyOWL007      7   
2  post  Meditation           kikemeister      2   
3  post  Meditation              TinaTeng      9   
4  post  Meditation                sidgat      8   
5  post  Meditation  Intrepid_Strike_2454      2   
6  post  Meditation    ExperienceMany4417     35   
7  post  Meditation         EthanGisclair      6   
8  post  Meditation        AnnualPath9528      5   
9  post  Meditation         Teastainedeye      8   

                                                                                                       title  \
0                                                 Equanimity and Non-Attachment Meditation, A Novel Approach   
1                                       I didn’t think meditation could help the planet… until I tried this.   
2                                                                                          

**Section 3. LLMs: Summary, key themes, pain points, and emotions**

In [5]:
!pip install -q transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.0 MB/s eta 0:00:00


In [6]:
!apt install -y libbitsandbytes-dev
!pip install bitsandbytes==0.41.1 --no-cache-dir --force-reinstall


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libbitsandbytes-dev
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 167.0 MB/s eta 0:00:00


In [ ]:
import os
os.kill(os.getpid(), 9)


In [7]:
import bitsandbytes as bnb
print("✅ bitsandbytes version:", bnb.__version__)


False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
The following directories listed in your path were found to be non-existent: {PosixPath('//mp.kaggle.net'), PosixPath('https')}
The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-a100-s-r59u71i66np --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=true ')}
The following 

/usr/local/lib/python3.11/dist-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


RuntimeError: 
        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [9]:
TEMPLATE = """
Carefully analyze the following user-generated content. Focus only on language related to meditation, mindfulness, emotional states, or experiences involving flow, awareness, or self-regulation — including any occasions, motivations, or purposes behind engaging in these practices.
Read the text below and complete the 9 tasks that follow.
---
{text}
---

Perform the following tasks with attention to the user’s original language and context:

Tasks:
1. Summarize only the parts related to meditation, mindfulness, flow, or self-regulation.
2. List user’s pain points using their exact words (no rephrasing).
3. List emotional expressions using the user's original words.
4. Give the overall emotion in 1 word (e.g., “anxious”, “calm”).
5. Give a sentiment score from -1 (negative) to +1 (positive).
6. Label sentiment as "positive" or "negative".
7. Identify the single most prominent theme in 1–3 words(e.g., "letting go", "inner peace").
8. Infer a likely demographic identity ("unknown" if unclear).
9. List phrases from the text that led to your demographic guess.


Return the answer in **exactly this JSON format**:

{{
  "summary": "...",
  "pain_points": ["..."],
  "emotion_phrases": ["..."],
  "emotion_label": "...",
  "sentiment_score": 0.42,
  "sentiment": "positive",
  "themes": ["..."],
  "demographic_proximity": "...",
  "demographic_evidence_phrases": ["..."]
}}
"""


In [10]:
import duckdb
import pandas as pd

# === ✅ Step 1: Load 1 Record from DuckDB ===
db_path = "/content/drive/MyDrive/reddit_data/reddit_20250605_083205.duckdb"  # update if needed
con = duckdb.connect(db_path)

df_loaded = con.execute("SELECT * FROM reddit_meditation_au").df()
con.close()



In [11]:
import json
import re
import torch

def fix_json_keys(raw_str):
    """
    Fixes common issues like unquoted keys in JSON-like text.
    """
    return re.sub(r'(?<={|,)\s*(\w+)\s*:', r'"\1":', raw_str)

def extract_last_json_block(output_text):
    """
    Extracts the last {...} block from the model output — the actual answer.
    """
    matches = list(re.finditer(r'\{[\s\S]*?\}', output_text))
    if not matches:
        return None
    return matches[-1].group(0)

def safe_generate_and_parse(input_text, tokenizer, model, template, max_tokens=512):
    """
    Generates a response from the model and safely parses it as structured JSON.
    """
    # Step 1: Insert input into template
    prompt = template.replace("{text}", input_text.strip())

    # Step 2: Tokenize prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    # Step 3: Generate output
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,  # deterministic
            use_cache=True
        )

    # Step 4: Decode model output
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Step 5: Extract the last valid JSON block (ignores formatting examples)
    json_str = extract_last_json_block(decoded_output)
    if not json_str:
        print("❌ Could not extract JSON block.")
        print("Raw output:\n", decoded_output)
        return None

    # Step 6: Try to parse raw or fixed JSON
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        try:
            fixed_str = fix_json_keys(json_str)
            return json.loads(fixed_str)
        except json.JSONDecodeError as e:
            print("❌ JSON still invalid after fix:", e)
            print("🧾 Final attempted string:\n", fixed_str)
            return None

In [12]:
!pip install tqdm

In [ ]:
import json
import pandas as pd
from tqdm import tqdm
import duckdb
from datetime import datetime
import os
import shutil
from google.colab import drive

# --- Config ---
BATCH_SIZE = 100
TABLE_NAME = "reddit_nlp_features"
LOCAL_DB_FILE = "reddit_progress.duckdb"
DRIVE_SUBFOLDER = "/content/drive/MyDrive/reddit_data"
DRIVE_DB_FILE = os.path.join(DRIVE_SUBFOLDER, LOCAL_DB_FILE)

# --- Mount Google Drive ---
drive.mount('/content/drive')
os.makedirs(DRIVE_SUBFOLDER, exist_ok=True)

# --- Validate Input ---
if "full_text" not in df_loaded.columns:
    raise ValueError("❌ 'full_text' column is missing from df_loaded.")

# --- Initialize DuckDB Connection ---
conn = duckdb.connect(LOCAL_DB_FILE)

# --- Batch Processing Loop ---
for start_idx in tqdm(range(0, len(df_loaded), BATCH_SIZE), desc="🔄 Batch Progress", unit="batch"):
    end_idx = min(start_idx + BATCH_SIZE, len(df_loaded))
    print(f"\n🚀 Processing batch: {start_idx}–{end_idx - 1}")

    batch = df_loaded.iloc[start_idx:end_idx]
    combined_rows = []

    for idx, row in batch.iterrows():
        try:
            input_text = str(row.get("full_text", ""))[:1800]
            parsed = safe_generate_and_parse(
                input_text=input_text,
                tokenizer=tokenizer,
                model=model,
                template=TEMPLATE
            )
            if parsed:
                parsed["row_index"] = idx
                row_meta = row.drop(labels=["full_text"]).to_dict()
                combined = {**row_meta, **parsed}

                for k, v in combined.items():
                    if hasattr(v, "tolist"):
                        combined[k] = v.tolist()
                    if isinstance(combined[k], list) and all(isinstance(item, str) for item in combined[k]):
                        combined[k] = "; ".join(combined[k])

                combined_rows.append(combined)
            else:
                print(f"⚠️ Row {idx} skipped: model returned no valid JSON.")
        except Exception as e:
            print(f"❌ Error on row {idx}: {e}")

    # --- Append to DuckDB ---
    if combined_rows:
        df_combined = pd.DataFrame(combined_rows)
        conn.execute(f"CREATE TABLE IF NOT EXISTS {TABLE_NAME} AS SELECT * FROM df_combined LIMIT 0")
        conn.execute(f"INSERT INTO {TABLE_NAME} SELECT * FROM df_combined")
        print(f"✅ Appended batch {start_idx}–{end_idx - 1} to DuckDB")

        # --- Copy to Google Drive (overwrite with full DB each time) ---
        shutil.copy(LOCAL_DB_FILE, DRIVE_DB_FILE)
        print(f"📁 Synced cumulative DB to Google Drive → {DRIVE_DB_FILE}")

# --- Finalize ---
conn.close()
print(f"\n🏁 All {len(df_loaded)} records processed and synced.")


**Breakdown for production**

In [2]:
import json
import pandas as pd
from tqdm import tqdm
import duckdb
from datetime import datetime
import os
import shutil
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from google.colab import drive

# --- Config ---
BATCH_SIZE = 100
TABLE_NAME = "reddit_nlp_features"
LOCAL_DB_FILE = "reddit_progress.duckdb"
DRIVE_SUBFOLDER = "/content/drive/MyDrive/reddit_data"
DRIVE_DB_FILE = os.path.join(DRIVE_SUBFOLDER, LOCAL_DB_FILE)

In [3]:
# ✅ Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# ✅ Step 2: Set full path to your DuckDB file
import duckdb

dest_file = "/content/drive/MyDrive/reddit_data/reddit_20250605_083205.duckdb"

# Connect to the file
con = duckdb.connect(dest_file)
# ✅ Step 3: Check available tables
tables = con.execute("SHOW TABLES").fetchall()
print("📋 Tables in Drive copy:", tables)

Mounted at /content/drive
📋 Tables in Drive copy: [('reddit_meditation_au',)]


In [4]:
# Try loading
df_loaded = con.execute("SELECT * FROM reddit_meditation_au").df()
print(f"✅ Loaded {len(df_loaded)} rows from Drive copy.")
con.close()
# --- Validate Input ---
if "full_text" not in df_loaded.columns:
    raise ValueError("❌ 'full_text' column is missing from df_loaded.")

✅ Loaded 2977 rows from Drive copy.


**GoEmotions is a corpus of 58k carefully curated comments extracted from Reddit, with human annotations to 27 emotion categories, along with label Neutral.**

References:
*   GoEmotions Dataset on Hugging Face: https://huggingface.co/datasets/google-research-datasets/go_emotions
*   Google Research Blog: GoEmotions: A Dataset for Fine-Grained Emotion Classification(https://research.google/blog/goemotions-a-dataset-for-fine-grained-emotion-classification/)
*  Original Paper: GoEmotions: A Dataset of Fine-Grained Emotions
(https://arxiv.org/abs/2005.00547)


For a comprehensive list of the emotion labels and additional details, you can refer to the dataset's README on Hugging Face: https://huggingface.co/datasets/google-research-datasets/go_emotions/blob/main/README.md


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# --- Emotion Inference Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
emotion_model_id = "joeddav/distilbert-base-uncased-go-emotions-student"
emotion_tokenizer = AutoTokenizer.from_pretrained(emotion_model_id)
emotion_model = AutoModelForSequenceClassification.from_pretrained(emotion_model_id).to(device)
emotion_model.eval()
emotion_labels = [
    "admiration", "amusement", "anger", "annoyance", "approval", "caring", "confusion", "curiosity",
    "desire", "disappointment", "disapproval", "disgust", "embarrassment", "excitement", "fear",
    "gratitude", "grief", "joy", "love", "nervousness", "optimism", "pride", "realization", "relief",
    "remorse", "sadness", "surprise", "neutral"
]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/421 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.75k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# --- LLM Setup (OpenHermes-2.5-Mistral-7B) ---
mistral_model_id = "teknium/OpenHermes-2.5-Mistral-7B"
llm_tokenizer = AutoTokenizer.from_pretrained(mistral_model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    mistral_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
llm_model.eval()

tokenizer_config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [7]:
TEMPLATE = """
Carefully analyze the following user-generated content. Focus only on language related to meditation, mindfulness, emotional states, or experiences involving flow, awareness, or self-regulation — including any occasions, motivations, or purposes behind engaging in these practices.
Read the text below and complete the 9 tasks that follow.
---
{text}
---

Perform the following tasks with attention to the user’s original language and context:

Tasks:
1. Summarize only the parts related to meditation, mindfulness, flow, or self-regulation.
2. List user’s pain points using their exact words (no rephrasing).
3. List emotional expressions using the user's original words.
4. Give the overall emotion in 1 word (e.g., “anxious”, “calm”).
5. Give a sentiment score from -1 (negative) to +1 (positive).
6. Label sentiment as "positive" or "negative".
7. Identify the single most prominent theme in 1–3 words(e.g., "letting go", "inner peace").
8. Infer a likely demographic identity ("unknown" if unclear).
9. List phrases from the text that led to your demographic guess.


Return the answer in **exactly this JSON format**:

{{
  "summary": "...",
  "pain_points": ["..."],
  "emotion_phrases": ["..."],
  "emotion_label": "...",
  "sentiment_score": 0.42,
  "sentiment": "positive",
  "themes": ["..."],
  "demographic_proximity": "...",
  "demographic_evidence_phrases": ["..."]
}}
"""

In [8]:
import json
import re
import torch
import time
from typing import Optional

# --- Global GPU-safe fallback constants ---
_JSON_FAIL_RESPONSE = {"success": False, "data": None, "error": "JSON parse failed"}
_EMOTION_FAIL_RESPONSE = lambda text: {"success": False, "data": dict(zip(emotion_labels, [None]*len(emotion_labels))), "error": "Emotion model failure", "text": text}

# --- JSON utility: fix common key issues ---
def fix_json_keys(raw_str):
    return re.sub(r'(?<={|,)\s*(\w+)\s*:', r'"\1":', raw_str)

def extract_last_json_block(output_text):
    matches = list(re.finditer(r'\{[\s\S]*?\}', output_text))
    return matches[-1].group(0) if matches else None

# --- Retry decorator for fault-tolerant inference ---
def retry_on_fail(max_retries=1, wait_seconds=1.0):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for attempt in range(max_retries + 1):
                try:
                    return func(*args, **kwargs)
                except torch.cuda.OutOfMemoryError as e:
                    print(f"🔥 GPU OOM on attempt {attempt + 1}: {e}")
                    torch.cuda.empty_cache()
                    time.sleep(wait_seconds)
                except Exception as e:
                    print(f"❌ {func.__name__} failed on attempt {attempt + 1}: {e}")
                    time.sleep(wait_seconds)
            return None
        return wrapper
    return decorator

# --- Robust LLM JSON parser ---
@retry_on_fail(max_retries=1)
def safe_generate_and_parse(input_text: str, tokenizer, model, template: str, max_tokens=512, row_id: Optional[int] = None):
    prompt = template.replace("{text}", input_text.strip())
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,
            use_cache=True
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    json_str = extract_last_json_block(decoded)

    if not json_str:
        print(f"❌ [{row_id}] No JSON block found.\nRaw output:\n{decoded[:300]}")
        return _JSON_FAIL_RESPONSE | {"raw": decoded, "row_id": row_id}

    try:
        parsed = json.loads(json_str)
        return {"success": True, "data": parsed, "row_id": row_id}
    except json.JSONDecodeError:
        try:
            fixed_str = fix_json_keys(json_str)
            parsed = json.loads(fixed_str)
            return {"success": True, "data": parsed, "row_id": row_id}
        except json.JSONDecodeError as e:
            print(f"❌ [{row_id}] JSON invalid after fix: {e}")
            return _JSON_FAIL_RESPONSE | {"raw": fixed_str, "row_id": row_id}

# --- Emotion classifier (GoEmotions) ---
@retry_on_fail(max_retries=1)
def get_emotion_vector(text: str, tokenizer, model, labels, device="cuda"):
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
            probs = torch.sigmoid(logits)[0].detach().cpu().numpy()
        emotion_dict = dict(zip(labels, probs))
        return {"success": True, "data": emotion_dict, "text": text}
    except Exception as e:
        print(f"❌ Emotion model error: {e}")
        return _EMOTION_FAIL_RESPONSE(text)


In [37]:
# --- DuckDB Connection ---
conn = duckdb.connect(LOCAL_DB_FILE)

# Optional: collect failed rows for inspection
failed_rows = []

# --- Main Batch Loop ---
for start_idx in tqdm(range(0, len(df_loaded), BATCH_SIZE), desc="🔄 Batch Progress", unit="batch"):
    end_idx = min(start_idx + BATCH_SIZE, len(df_loaded))
    print(f"\n🚀 Processing batch: {start_idx}–{end_idx - 1}")

    batch = df_loaded.iloc[start_idx:end_idx]
    combined_rows = []

    for idx, row in batch.iterrows():
        try:
            text = str(row.get("full_text", ""))[:1800]
            row_meta = row.drop(labels=["full_text"]).to_dict()

            # 1. Run LLM parser
            parsed = safe_generate_and_parse(
                input_text=text,
                tokenizer=llm_tokenizer,
                model=llm_model,
                template=TEMPLATE,
                row_id=idx
            )

            # 2. Run emotion model
            emotion = get_emotion_vector(
                text=text,
                tokenizer=emotion_tokenizer,
                model=emotion_model,
                labels=emotion_labels,
                device=device
            )

            if parsed["success"] and emotion["success"]:
                # 3. Merge results
                combined = {
                    **row_meta,
                    **parsed["data"],
                    **emotion["data"],
                    "row_index": idx,
                    "text": text
                }

                # 4. Flatten for table compatibility
                for k, v in combined.items():
                    if hasattr(v, "tolist"):
                        combined[k] = v.tolist()
                    if isinstance(combined[k], list) and all(isinstance(item, str) for item in combined[k]):
                        combined[k] = "; ".join(combined[k])

                combined_rows.append(combined)
            else:
                print(f"⚠️ Row {idx} skipped: JSON or emotion parse failed.")
                failed_rows.append({
                    "row_index": idx,
                    "error": parsed.get("error") or emotion.get("error"),
                    "text": text[:300]
                })

        except Exception as e:
            print(f"❌ Error on row {idx}: {e}")
            failed_rows.append({
                "row_index": idx,
                "error": str(e),
                "text": text[:300]
            })

    # --- Save batch results ---
    if combined_rows:
        df_combined = pd.DataFrame(combined_rows)
        conn.execute(f"CREATE TABLE IF NOT EXISTS {TABLE_NAME} AS SELECT * FROM df_combined LIMIT 0")
        conn.execute(f"INSERT INTO {TABLE_NAME} SELECT * FROM df_combined")
        print(f"✅ Appended batch {start_idx}–{end_idx - 1} to DuckDB")

        shutil.copy(LOCAL_DB_FILE, DRIVE_DB_FILE)
        print(f"📁 Synced cumulative DB to Google Drive → {DRIVE_DB_FILE}")

# --- Finalize ---
conn.close()

# --- Save failed rows log (optional) ---
if failed_rows:
    pd.DataFrame(failed_rows).to_csv("failed_rows.csv", index=False)
    print(f"⚠️ Logged {len(failed_rows)} failed rows to failed_rows.csv")

print(f"\n🏁 All {len(df_loaded)} records processed and synced.")

🔄 Batch Progress:   0%|          | 0/30 [00:00<?, ?batch/s]


🚀 Processing batch: 0–99
❌ [62] JSON invalid after fix: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
⚠️ Row 62 skipped: JSON or emotion parse failed.


🔄 Batch Progress:   3%|▎         | 1/30 [16:58<8:12:12, 1018.37s/batch]

✅ Appended batch 0–99 to DuckDB
📁 Synced cumulative DB to Google Drive → /content/drive/MyDrive/reddit_data/reddit_progress.duckdb

🚀 Processing batch: 100–199
❌ [126] JSON invalid after fix: Invalid \escape: line 3 column 125 (char 321)
⚠️ Row 126 skipped: JSON or emotion parse failed.


🔄 Batch Progress:   7%|▋         | 2/30 [32:28<7:30:53, 966.20s/batch] 

...

✅ Appended batch 800–899 to DuckDB
📁 Synced cumulative DB to Google Drive → /content/drive/MyDrive/reddit_data/reddit_progress.duckdb

🚀 Processing batch: 900–999
❌ [943] JSON invalid after fix: Expecting ',' delimiter: line 10 column 41 (char 682)
⚠️ Row 943 skipped: JSON or emotion parse failed.
❌ [993] JSON invalid after fix: Expecting value: line 4 column 286 (char 551)
⚠️ Row 993 skipped: JSON or emotion parse failed.


🔄 Batch Progress:  33%|███▎      | 10/30 [2:39:50<5:18:41, 956.09s/batch]

✅ Appended batch 900–999 to DuckDB
📁 Synced cumulative DB to Google Drive → /content/drive/MyDrive/reddit_data/reddit_progress.duckdb

🚀 Processing batch: 1000–1099


🔄 Batch Progress:  33%|███▎      | 10/30 [2:54:11<5:48:23, 1045.15s/batch]


BinderException: Binder Error: table reddit_nlp_features has 51 columns but 52 values were supplied

**for start_idx in tqdm(range(1000, **
replace with 0 once complete to reset

In [40]:
# --- DuckDB Connection ---
conn = duckdb.connect(LOCAL_DB_FILE)

# Optional: collect failed rows for inspection
failed_rows = []

# --- Main Batch Loop ---
for start_idx in tqdm(range(1000, len(df_loaded), BATCH_SIZE), desc="🔄 Batch Progress", unit="batch"):
    end_idx = min(start_idx + BATCH_SIZE, len(df_loaded))
    print(f"\n🚀 Processing batch: {start_idx}–{end_idx - 1}")

    batch = df_loaded.iloc[start_idx:end_idx]
    combined_rows = []

    for idx, row in batch.iterrows():
        try:
            text = str(row.get("full_text", ""))[:1800]
            row_meta = row.drop(labels=["full_text"]).to_dict()

            # 1. Run LLM parser
            parsed = safe_generate_and_parse(
                input_text=text,
                tokenizer=llm_tokenizer,
                model=llm_model,
                template=TEMPLATE,
                row_id=idx
            )

            # 2. Run emotion model
            emotion = get_emotion_vector(
                text=text,
                tokenizer=emotion_tokenizer,
                model=emotion_model,
                labels=emotion_labels,
                device=device
            )

            if parsed["success"] and emotion["success"]:
                # 3. Merge results
                combined = {
                    **row_meta,
                    **parsed["data"],
                    **emotion["data"],
                    "row_index": idx,
                    "text": text
                }

                # 4. Flatten for table compatibility
                for k, v in combined.items():
                    if hasattr(v, "tolist"):
                        combined[k] = v.tolist()
                    if isinstance(combined[k], list) and all(isinstance(item, str) for item in combined[k]):
                        combined[k] = "; ".join(combined[k])

                combined_rows.append(combined)
            else:
                print(f"⚠️ Row {idx} skipped: JSON or emotion parse failed.")
                failed_rows.append({
                    "row_index": idx,
                    "error": parsed.get("error") or emotion.get("error"),
                    "text": text[:300]
                })

        except Exception as e:
            print(f"❌ Error on row {idx}: {e}")
            failed_rows.append({
                "row_index": idx,
                "error": str(e),
                "text": text[:300]
            })

    # --- Save batch results ---
    if combined_rows:
        df_combined = pd.DataFrame(combined_rows)
        conn.execute(f"CREATE TABLE IF NOT EXISTS {TABLE_NAME} AS SELECT * FROM df_combined LIMIT 0")
        conn.execute(f"INSERT INTO {TABLE_NAME} SELECT * FROM df_combined")
        print(f"✅ Appended batch {start_idx}–{end_idx - 1} to DuckDB")

        shutil.copy(LOCAL_DB_FILE, DRIVE_DB_FILE)
        print(f"📁 Synced cumulative DB to Google Drive → {DRIVE_DB_FILE}")

# --- Finalize ---
conn.close()

# --- Save failed rows log (optional) ---
if failed_rows:
    pd.DataFrame(failed_rows).to_csv("failed_rows.csv", index=False)
    print(f"⚠️ Logged {len(failed_rows)} failed rows to failed_rows.csv")

print(f"\n🏁 All {len(df_loaded)} records processed and synced.")

🔄 Batch Progress:   0%|          | 0/20 [00:00<?, ?batch/s]


🚀 Processing batch: 1000–1099


🔄 Batch Progress:   0%|          | 0/20 [14:19<?, ?batch/s]


BinderException: Binder Error: table reddit_nlp_features has 51 columns but 52 values were supplied

In [ ]:
# --- DuckDB Connection ---
conn = duckdb.connect(LOCAL_DB_FILE)

# Optional: fetch schema if table exists
try:
    table_cols = [col[1] for col in conn.execute(f"PRAGMA table_info({TABLE_NAME})").fetchall()]
except:
    table_cols = []

# Optional: collect failed rows for inspection
failed_rows = []

# --- Main Batch Loop ---
for start_idx in tqdm(range(1000, len(df_loaded), BATCH_SIZE), desc="🔄 Batch Progress", unit="batch"):
    end_idx = min(start_idx + BATCH_SIZE, len(df_loaded))
    print(f"\n🚀 Processing batch: {start_idx}–{end_idx - 1}")

    batch = df_loaded.iloc[start_idx:end_idx]
    combined_rows = []

    for idx, row in batch.iterrows():
        try:
            text = str(row.get("full_text", ""))[:1800]
            row_meta = row.drop(labels=["full_text"]).to_dict()

            # 1. Run LLM parser
            parsed = safe_generate_and_parse(
                input_text=text,
                tokenizer=llm_tokenizer,
                model=llm_model,
                template=TEMPLATE,
                row_id=idx
            )

            # 2. Run emotion model
            emotion = get_emotion_vector(
                text=text,
                tokenizer=emotion_tokenizer,
                model=emotion_model,
                labels=emotion_labels,
                device=device
            )

            if parsed["success"] and emotion["success"]:
                # 3. Merge results
                combined = {
                    **row_meta,
                    **parsed["data"],
                    **emotion["data"],
                    "row_index": idx,
                    "text": text
                }

                # 4. Flatten for table compatibility
                for k, v in combined.items():
                    if hasattr(v, "tolist"):
                        combined[k] = v.tolist()
                    if isinstance(combined[k], list) and all(isinstance(item, str) for item in combined[k]):
                        combined[k] = "; ".join(combined[k])

                combined_rows.append(combined)
            else:
                print(f"⚠️ Row {idx} skipped: JSON or emotion parse failed.")
                failed_rows.append({
                    "row_index": idx,
                    "error": parsed.get("error") or emotion.get("error"),
                    "text": text[:300]
                })

        except Exception as e:
            print(f"❌ Error on row {idx}: {e}")
            failed_rows.append({
                "row_index": idx,
                "error": str(e),
                "text": text[:300]
            })

    # --- Save batch results ---
    if combined_rows:
        df_combined = pd.DataFrame(combined_rows)
        # Align columns
        if table_cols:
            df_combined = df_combined[[col for col in df_combined.columns if col in table_cols]]
        else:
            # First batch defines table schema
            table_cols = df_combined.columns.tolist()
            conn.execute(f"CREATE TABLE IF NOT EXISTS {TABLE_NAME} AS SELECT * FROM df_combined LIMIT 0")

        # Append to table
        conn.execute(f"INSERT INTO {TABLE_NAME} SELECT * FROM df_combined")
        print(f"✅ Appended batch {start_idx}–{end_idx - 1} to DuckDB")

        # Sync to Drive
        shutil.copy(LOCAL_DB_FILE, DRIVE_DB_FILE)
        print(f"📁 Synced cumulative DB to Google Drive → {DRIVE_DB_FILE}")

# --- Finalize ---
conn.close()

# --- Save failed rows log (optional) ---
if failed_rows:
    pd.DataFrame(failed_rows).to_csv("failed_rows.csv", index=False)
    print(f"⚠️ Logged {len(failed_rows)} failed rows to failed_rows.csv")

print(f"\n🏁 All {len(df_loaded)} records processed and synced.")

🔄 Batch Progress:   0%|          | 0/20 [00:00<?, ?batch/s]


🚀 Processing batch: 1000–1099


🔄 Batch Progress:   5%|▌         | 1/20 [14:20<4:32:23, 860.16s/batch]

✅ Appended batch 1000–1099 to DuckDB
📁 Synced cumulative DB to Google Drive → /content/drive/MyDrive/reddit_data/reddit_progress.duckdb

🚀 Processing batch: 1100–1199
❌ [1159] JSON invalid after fix: Expecting ',' delimiter: line 4 column 74 (char 294)
⚠️ Row 1159 skipped: JSON or emotion parse failed.


🔄 Batch Progress:  10%|█         | 2/20 [28:53<4:20:22, 867.90s/batch]

...

✅ Appended batch 2000–2099 to DuckDB
📁 Synced cumulative DB to Google Drive → /content/drive/MyDrive/reddit_data/reddit_progress.duckdb

🚀 Processing batch: 2100–2199
❌ [2156] JSON invalid after fix: Invalid \escape: line 4 column 124 (char 610)
⚠️ Row 2156 skipped: JSON or emotion parse failed.
❌ [2162] JSON invalid after fix: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
⚠️ Row 2162 skipped: JSON or emotion parse failed.
❌ [2182] JSON invalid after fix: Expecting property name enclosed in double quotes: line 11 column 1 (char 1166)
⚠️ Row 2182 skipped: JSON or emotion parse failed.


🔄 Batch Progress:  60%|██████    | 12/20 [3:04:05<2:05:20, 940.02s/batch]

✅ Appended batch 2100–2199 to DuckDB
📁 Synced cumulative DB to Google Drive → /content/drive/MyDrive/reddit_data/reddit_progress.duckdb

🚀 Processing batch: 2200–2299
❌ [2249] JSON invalid after fix: Expecting property name enclosed in double quotes: line 11 column 1 (char 803)
⚠️ Row 2249 skipped: JSON or emotion parse failed.


🔄 Batch Progress:  65%|██████▌   | 13/20 [3:19:59<1:50:09, 944.24s/batch]

✅ Appended batch 2200–2299 to DuckDB
📁 Synced cumulative DB to Google Drive → /content/drive/MyDrive/reddit_data/reddit_progress.duckdb

🚀 Processing batch: 2300–2399


In [9]:
# --- DuckDB Connection ---
conn = duckdb.connect(LOCAL_DB_FILE)

# Optional: fetch schema if table exists
try:
    table_cols = [col[1] for col in conn.execute(f"PRAGMA table_info({TABLE_NAME})").fetchall()]
except:
    table_cols = []

# Optional: collect failed rows for inspection
failed_rows = []

# --- Main Batch Loop ---
for start_idx in tqdm(range(2300, len(df_loaded), BATCH_SIZE), desc="🔄 Batch Progress", unit="batch"):
    end_idx = min(start_idx + BATCH_SIZE, len(df_loaded))
    print(f"\n🚀 Processing batch: {start_idx}–{end_idx - 1}")

    batch = df_loaded.iloc[start_idx:end_idx]
    combined_rows = []

    for idx, row in batch.iterrows():
        try:
            text = str(row.get("full_text", ""))[:1800]
            row_meta = row.drop(labels=["full_text"]).to_dict()

            # 1. Run LLM parser
            parsed = safe_generate_and_parse(
                input_text=text,
                tokenizer=llm_tokenizer,
                model=llm_model,
                template=TEMPLATE,
                row_id=idx
            )

            # 2. Run emotion model
            emotion = get_emotion_vector(
                text=text,
                tokenizer=emotion_tokenizer,
                model=emotion_model,
                labels=emotion_labels,
                device=device
            )

            if parsed["success"] and emotion["success"]:
                # 3. Merge results
                combined = {
                    **row_meta,
                    **parsed["data"],
                    **emotion["data"],
                    "row_index": idx,
                    "text": text
                }

                # 4. Flatten for table compatibility
                for k, v in combined.items():
                    if hasattr(v, "tolist"):
                        combined[k] = v.tolist()
                    if isinstance(combined[k], list) and all(isinstance(item, str) for item in combined[k]):
                        combined[k] = "; ".join(combined[k])

                combined_rows.append(combined)
            else:
                print(f"⚠️ Row {idx} skipped: JSON or emotion parse failed.")
                failed_rows.append({
                    "row_index": idx,
                    "error": parsed.get("error") or emotion.get("error"),
                    "text": text[:300]
                })

        except Exception as e:
            print(f"❌ Error on row {idx}: {e}")
            failed_rows.append({
                "row_index": idx,
                "error": str(e),
                "text": text[:300]
            })

    # --- Save batch results ---
    if combined_rows:
        df_combined = pd.DataFrame(combined_rows)
        # Align columns
        if table_cols:
            df_combined = df_combined[[col for col in df_combined.columns if col in table_cols]]
        else:
            # First batch defines table schema
            table_cols = df_combined.columns.tolist()
            conn.execute(f"CREATE TABLE IF NOT EXISTS {TABLE_NAME} AS SELECT * FROM df_combined LIMIT 0")

        # Append to table
        conn.execute(f"INSERT INTO {TABLE_NAME} SELECT * FROM df_combined")
        print(f"✅ Appended batch {start_idx}–{end_idx - 1} to DuckDB")

        # Sync to Drive
        shutil.copy(LOCAL_DB_FILE, DRIVE_DB_FILE)
        print(f"📁 Synced cumulative DB to Google Drive → {DRIVE_DB_FILE}")

# --- Finalize ---
conn.close()

# --- Save failed rows log (optional) ---
if failed_rows:
    pd.DataFrame(failed_rows).to_csv("failed_rows.csv", index=False)
    print(f"⚠️ Logged {len(failed_rows)} failed rows to failed_rows.csv")

print(f"\n🏁 All {len(df_loaded)} records processed and synced.")

🔄 Batch Progress:   0%|          | 0/7 [00:00<?, ?batch/s]


🚀 Processing batch: 2300–2399
✅ Appended batch 2300–2399 to DuckDB


🔄 Batch Progress:  14%|█▍        | 1/7 [14:24<1:26:29, 864.90s/batch]

📁 Synced cumulative DB to Google Drive → /content/drive/MyDrive/reddit_data/reddit_progress.duckdb

🚀 Processing batch: 2400–2499
❌ [2459] JSON invalid after fix: Expecting ',' delimiter: line 3 column 51 (char 179)
⚠️ Row 2459 skipped: JSON or emotion parse failed.


🔄 Batch Progress:  29%|██▊       | 2/7 [30:02<1:15:39, 907.89s/batch]

...

✅ Appended batch 2700–2799 to DuckDB
📁 Synced cumulative DB to Google Drive → /content/drive/MyDrive/reddit_data/reddit_progress.duckdb

🚀 Processing batch: 2800–2899


🔄 Batch Progress:  86%|████████▌ | 6/7 [1:32:07<15:21, 921.70s/batch]

✅ Appended batch 2800–2899 to DuckDB
📁 Synced cumulative DB to Google Drive → /content/drive/MyDrive/reddit_data/reddit_progress.duckdb

🚀 Processing batch: 2900–2976


🔄 Batch Progress: 100%|██████████| 7/7 [1:44:10<00:00, 892.97s/batch]

✅ Appended batch 2900–2976 to DuckDB
📁 Synced cumulative DB to Google Drive → /content/drive/MyDrive/reddit_data/reddit_progress.duckdb
⚠️ Logged 6 failed rows to failed_rows.csv

🏁 All 2977 records processed and synced.


In [38]:
print("DuckDB expected columns:", conn.execute(f"PRAGMA table_info({TABLE_NAME})").fetchall())
print("DataFrame columns:", df_combined.columns.tolist())


DuckDB expected columns: [(0, 'type', 'VARCHAR', False, None, False), (1, 'subreddit', 'VARCHAR', False, None, False), (2, 'author', 'VARCHAR', False, None, False), (3, 'score', 'BIGINT', False, None, False), (4, 'title', 'VARCHAR', False, None, False), (5, 'matched_terms', 'VARCHAR', False, None, False), (6, 'url', 'VARCHAR', False, None, False), (7, 'created_utc', 'VARCHAR', False, None, False), (8, 'link_flair_text', 'VARCHAR', False, None, False), (9, 'num_comments', 'DOUBLE', False, None, False), (10, 'id', 'VARCHAR', False, None, False), (11, 'comment_depth', 'DOUBLE', False, None, False), (12, 'summary', 'VARCHAR', False, None, False), (13, 'pain_points', 'VARCHAR', False, None, False), (14, 'emotion_phrases', 'VARCHAR', False, None, False), (15, 'emotion_label', 'VARCHAR', False, None, False), (16, 'sentiment_score', 'DOUBLE', False, None, False), (17, 'sentiment', 'VARCHAR', False, None, False), (18, 'themes', 'VARCHAR', False, None, False), (19, 'demographic_proximity', 'VARC

In [20]:
pd.set_option('display.max_columns', None)
df_combined

,type,subreddit,author,score,title,matched_terms,url,created_utc,link_flair_text,num_comments,id,comment_depth,summary,pain_points,emotion_phrases,emotion_label,sentiment_score,sentiment,themes,demographic_proximity,demographic_evidence_phrases,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,row_index,text
0,post,Meditation,Anima_Monday,2,"Equanimity and Non-Attachment Meditation, A No...",act,https://reddit.com/r/Meditation/comments/1l3o2...,2025-06-05T02:27:44,Sharing / Insight 💡,0.0,1l3o2jc,NaN,The user developed a meditation practice using...,none,fine,calm,0.57,positive,meditation practice; equanimity; non-attachment,unknown,none,0.430545,0.167192,0.182123,0.278183,0.653991,0.916768,0.486951,0.643949,0.64548,0.236684,0.442498,0.225125,0.265867,0.268055,0.285222,0.455119,0.294263,0.363282,0.502222,0.247318,0.316201,0.255882,0.692624,0.653467,0.410916,0.32566,0.338183,0.759731,0,This practice uses phrases for equanimity and ...


**Section 4. Reload Reddit Scraped Results from Colab**

In [ ]:
from google.colab import drive
import os
import duckdb
import pandas as pd

# --- Config ---
TABLE_NAME = "reddit_nlp_features"
LOCAL_DB_FILE = "reddit_progress.duckdb"
DRIVE_SUBFOLDER = "/content/drive/MyDrive/reddit_data"
DRIVE_DB_FILE = os.path.join(DRIVE_SUBFOLDER, LOCAL_DB_FILE)

DRIVE_DB_FILE = os.path.join(DRIVE_SUBFOLDER, LOCAL_DB_FILE)
# --- Mount Google Drive (if not already) ---
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- Connect to DB file in Drive ---
conn = duckdb.connect(DRIVE_DB_FILE)

# --- List tables ---
tables = conn.execute("SHOW TABLES").fetchall()
print("📋 Tables found:", tables)

# --- Check if target table exists ---
if (TABLE_NAME,) in tables:
    df_reddit = conn.execute(f"SELECT * FROM {TABLE_NAME}").df()
    print(f"✅ Loaded {len(df_reddit)} rows from table '{TABLE_NAME}'")
    pd.set_option('display.max_columns', None)
    display(df_reddit.head())
else:
    print(f"❌ Table '{TABLE_NAME}' not found in the database.")

conn.close()

In [ ]:
df_reddit.head()

**Section 5. Cluster Analysis**

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(page_title="Emotion Cluster Explorer", layout="wide")

# 📥 Load your processed PCA and UMAP clustering results
@st.cache_data
def load_data():
    pca_df = pd.read_csv("pca_clusters.csv")
    umap_df = pd.read_csv("umap_clusters.csv")
    return pca_df, umap_df

pca_df, umap_df = load_data()

# 🎯 Choose projection method
st.sidebar.title("Options")
projection = st.sidebar.radio("Choose projection", ["PCA", "UMAP"])
use_3d = st.sidebar.checkbox("3D Mode", value=True)

# 📊 Choose dataset
df = pca_df if projection == "PCA" else umap_df
x = f"{projection}1"
y = f"{projection}2"
z = f"{projection}3" if use_3d and f"{projection}3" in df.columns else None

# 📈 Generate interactive plot
st.title(f"🎨 {projection} Emotional Cluster Explorer")

if z:
    fig = px.scatter_3d(
        df,
        x=x, y=y, z=z,
        color="emotional_theme",
        hover_data=["full_text", "cluster"]
    )
else:
    fig = px.scatter(
        df,
        x=x, y=y,
        color="emotional_theme",
        hover_data=["full_text", "cluster"]
    )

st.plotly_chart(fig, use_container_width=True)

# 🧾 Optional: View text samples in a cluster
cluster_id = st.selectbox("🔍 View texts from cluster", sorted(df["cluster"].unique()))
st.dataframe(df[df["cluster"] == cluster_id][["full_text", "emotional_theme"]].head(30))


In [ ]:
# Install if needed:
# !pip install pandas scikit-learn umap-learn plotly

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import umap
import plotly.express as px

# --- Load Data ---
# Replace this with your actual dataset
df = pd.DataFrame({
    "themes": [
        "hope; confusion", "joy; pride", "grief; sadness", "anger; frustration",
        "hope; joy", "confusion; curiosity", "relief; pride", "sadness; love",
        "hope; grief", "joy; love"
    ]
})

# --- Step 1: Vectorize the 'themes' column ---
vectorizer = CountVectorizer(tokenizer=lambda x: x.split("; "))
X = vectorizer.fit_transform(df["themes"])  # binary indicator matrix

# --- Step 2: Reduce dimensions with PCA and UMAP ---
X_pca = PCA(n_components=2, random_state=42).fit_transform(X.toarray())
X_umap = umap.UMAP(n_components=2, random_state=42).fit_transform(X.toarray())

# --- Step 3: KMeans clustering on original feature space ---
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df["theme_cluster"] = kmeans.fit_predict(X.toarray())

# --- Step 4: Interactive Plot - PCA space ---
fig_pca = px.scatter(
    x=X_pca[:, 0], y=X_pca[:, 1],
    color=df["theme_cluster"].astype(str),
    hover_data={"themes": df["themes"]},
    title="🧠 Theme Clusters (PCA)"
)

# --- Step 5: Interactive Plot - UMAP space ---
fig_umap = px.scatter(
    x=X_umap[:, 0], y=X_umap[:, 1],
    color=df["theme_cluster"].astype(str),
    hover_data={"themes": df["themes"]},
    title="🌀 Theme Clusters (UMAP)"
)

# --- Step 6: Cluster Theme Profiles ---
theme_matrix = X.toarray()
df_themes = pd.DataFrame(theme_matrix, columns=vectorizer.get_feature_names_out())
cluster_profiles = df_themes.groupby(df["theme_cluster"]).mean().round(2)

# --- Display ---
print("\n📊 Mean theme presence per cluster:")
print(cluster_profiles)

# Show interactive plots
fig_pca.show()
fig_umap.show()


In [ ]:
# theme_cluster_analysis.py

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import umap
import plotly.express as px

def preprocess_features(df, feature_col="features"):
    """Tokenizes and vectorizes the features column."""
    vectorizer = CountVectorizer(tokenizer=lambda x: x.split("; "))
    X = vectorizer.fit_transform(df[feature_col])
    feature_names = vectorizer.get_feature_names_out()
    return X, feature_names

def run_dimensionality_reduction(X):
    """Runs PCA and UMAP in parallel."""
    X_pca = PCA(n_components=2, random_state=42).fit_transform(X.toarray())
    X_umap = umap.UMAP(n_components=2, random_state=42).fit_transform(X.toarray())
    return X_pca, X_umap

def run_kmeans_clustering(X, n_clusters=5):
    """Clusters the original vectorized features."""
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(X.toarray())
    return labels

def generate_cluster_profiles(X, labels, feature_names):
    """Generates average presence of each feature per cluster."""
    df_features = pd.DataFrame(X.toarray(), columns=feature_names)
    cluster_profiles = df_features.groupby(labels).mean().round(2)
    return cluster_profiles

def plot_clusters_2d(X_2d, labels, df, hover_columns, title="Cluster Plot"):
    """Plots a 2D scatter plot of the clusters."""
    hover_data = {col: df[col] for col in hover_columns if col in df.columns}
    fig = px.scatter(
        x=X_2d[:, 0], y=X_2d[:, 1],
        color=labels.astype(str),
        hover_data=hover_data,
        title=title
    )
    fig.show()

def run_feature_cluster_analysis(df, feature_col="features", n_clusters=5, hover_columns=["features"]):
    X, feature_names = preprocess_features(df, feature_col)
    X_pca, X_umap = run_dimensionality_reduction(X)
    labels = run_kmeans_clustering(X, n_clusters)
    df["feature_cluster"] = labels

    print("\n📊 Cluster Profiles:")
    profiles = generate_cluster_profiles(X, labels, feature_names)
    print(profiles)

    plot_clusters_2d(X_pca, labels, df, hover_columns, title="🧠 Feature Clusters (PCA)")
    plot_clusters_2d(X_umap, labels, df, hover_columns, title="🌀 Feature Clusters (UMAP)")

    return df, profiles


In [ ]:
df_result, profiles = run_feature_cluster_analysis(
    df,
    feature_col="themes",
    n_clusters=5,
    hover_columns=["themes", "source", "timestamp"]
)


**BACKUP Section 3. LLMs NLP(Alternative-Gemma 2)**

In [ ]:
from huggingface_hub import login
import os


from google.colab import userdata
hf_token = userdata.get('HuggingFace')

if hf_token is None:
    raise ValueError("🔐 Hugging Face token not found. Please add it to Colab secrets under 'HF_TOKEN'.")

# Authenticate with Hugging Face Hub
login(token=hf_token)

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install sentence-transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# === HuggingFace Model ID ===
model_id = "google/gemma-2b-it"  # or your preferred size like "gemma-7b-it"

# === Load Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token  # ensures padding is valid

# === Load Model (FP16, no quantization) ===
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    token=hf_token
)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaR